In [69]:
"""
For this experiment  you need to have your own csv containing types of questions that users can
ask or ask the AI team to for their dataset as we did not upload the csv to this repo
"""

'\nFor this experiment  you need to have your own csv containing types of questions that users can\nask or ask the AI team to for their dataset as we did not upload the csv to this repo\n'

In [70]:
import pandas as pd
from ai_docs_filterer_for_RAG import run_rm_labeller
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.openai import OpenAIProvider
from openai import AsyncAzureOpenAI
import os
import asyncio
from dotenv import load_dotenv
from ccs_website_data import  fetch_all_ccs_frameworks
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
import openai

load_dotenv()

True

In [71]:
test_data = pd.read_csv("query_filter_questions.csv")
ccs_frameworks = fetch_all_ccs_frameworks()
pydantic_azure_client = AsyncAzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_deployment=os.getenv("DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),

)
pydantic_rm_labeller_model = OpenAIChatModel(
    model_name= os.getenv("DEPLOYMENT_NAME"),
    provider=OpenAIProvider(openai_client=pydantic_azure_client),

)

rm_descriptions = "\n".join([
    f"RM: {r.rm_number} | Title: {r.title} | Status: {r.status}\n"
    f"Summary: {r.summary}\n"
    f"Benefits: {r.benefits if pd.notna(r.benefits) else 'N/A'}\n"
    f"Buying Guide: {r.how_to_buy if pd.notna(r.how_to_buy) else 'N/A'}\n"
    f"Context: {r.pillar} - {r.category} | Keywords: {r.keywords}\n"
    "---"
    for _, r in ccs_frameworks.iloc[:20].iterrows()
]) # Got an accuracy of 93.68%

# rm_descriptions = "\n".join([
#     f"RM: {r.rm_number} | "
#     f"Keywords: {r.keywords if 'keywords' in r and str(r.keywords).strip() else 'N/A'} | "
#     f"Summary: {r.summary} | "
#     f"Pillar: {r.pillar} ({r.category})"
#     for _, r in ccs_frameworks.iterrows()
# ])# Got an accuracy of 82%


Starting to fetch and clean frameworks (Status: Live,Expired)...
Fetched Page 1. Total frameworks collected: 268
Finished. Reached the last page: 1.

✅ DataFrame created successfully!


## Running experiment

In [77]:

# controls the concurrency only 3 rows at the time
sem = asyncio.Semaphore(3)

@retry(
    # exponentially increase wait time
    wait=wait_exponential(min=4, max=60),
    retry=retry_if_exception_type(openai.RateLimitError)
)
async def get_label_persistent(query_text, index):
    # time delay
    await asyncio.sleep(index * 0.5)

    async with sem:
        result = await run_rm_labeller(pydantic_rm_labeller_model, rm_descriptions, query_text)
        return result.rm_number

async def process_dataframe(df):
    # collect all conccurency tasks
    tasks = [get_label_persistent(row['query'], i) for i, row in df.iterrows()]

    # return_exceptions=True ensures one crash doesn't cancel the whole list
    results = await asyncio.gather(*tasks, return_exceptions=True)

    # Final check: if something still failed to manually re run experiment again
    df['predicted_rm'] = [
        res if not isinstance(res, Exception) else f"Error: {type(res).__name__}"
        for res in results
    ]

    return df

In [78]:
result_df = await process_dataframe(test_data)

In [74]:
print(result_df["predicted_rm"])

0     RM6348
1     RM6348
2     RM6348
3     RM6348
4     RM6348
       ...  
90    RM6292
91    RM6292
92    RM6292
93    RM6292
94    RM6292
Name: predicted_rm, Length: 95, dtype: object


In [79]:

accuracy = (result_df['Truth Set'] == result_df['predicted_rm']).mean()

print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 93.68%


In [80]:
mismatched_df = result_df[result_df['Truth Set'] != result_df['predicted_rm']]
print(mismatched_df)

                                                query Truth Set predicted_rm
14  How do I view the archived Apprenticeships and...    RM3823       RM6348
27  I need access to audit and assurance services ...    RM6188       RM6310
29         Is there a new agreement replacing RM6188?    RM6188       RM6310
43  Where can I find a wide range of cloud softwar...    RM6194       RM6285
44  Is RM6194 the current version of the back offi...    RM6194       RM6285
86  I need access to IaaS and PaaS through the exp...    RM6111       RM6292
